*Original code: Beating the Benchmark from West Nile Virus Prediction @ Kaggle by Abhihsek. Modified by Brendan Lane*

In [119]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D, MaxPooling1D
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import *
from keras.optimizers import Adam
from keras.initializers import *

%matplotlib inline

In [120]:
%%javascript
$('.nbp-app-bar').toggle()

<IPython.core.display.Javascript object>

In [197]:
# Load dataset 
train = pd.read_csv('../assets/train.csv')
test = pd.read_csv('../assets/test.csv')
sample = pd.read_csv('../assets/sampleSubmission.csv')
weather = pd.read_csv('../assets/weather.csv')

In [198]:
# Not using codesum for this benchmark
weather.drop('CodeSum', axis='columns', inplace=True)

In [199]:
# Split station 1 and 2 and join horizontally
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
weather = weather_stn1.merge(weather_stn2, on='Date')

In [200]:
# replace some missing values and T with -1
weather = weather.replace('M', -1)
weather = weather.replace('-', -1)
weather = weather.replace('T', -1)
weather = weather.replace(' T', -1)
weather = weather.replace('  T', -1)

In [201]:
# Functions to extract month and day from dataset
# You can also use parse_dates of Pandas.
def create_year(x):
    return x.split('-')[0]
def create_month(x):
    return x.split('-')[1]
def create_day(x):
    return x.split('-')[2]

train['month'] = train.Date.apply(create_month)
train['day'] = train.Date.apply(create_day)
train['year'] = train.Date.apply(create_year)

test['month'] = test.Date.apply(create_month)
test['day'] = test.Date.apply(create_day)
test['year'] = test.Date.apply(create_year)

In [202]:
# drop address columns
train = train.drop(['Address', 'AddressNumberAndStreet'], axis = 1)
test = test.drop(['Id', 'Address', 'AddressNumberAndStreet'], axis = 1)

In [203]:
# Merge with weather data
train = train.merge(weather, on='Date')
test = test.merge(weather, on='Date')
train = train.drop(['Date'], axis = 1)
test = test.drop(['Date'], axis = 1)

In [204]:
# Convert categorical data to numbers
lbl = LabelEncoder()
lbl.fit(list(train['Species'].values) + list(test['Species'].values))
train['Species'] = lbl.transform(train['Species'].values)
test['Species'] = lbl.transform(test['Species'].values)

lbl.fit(list(train['Street'].values) + list(test['Street'].values))
train['Street'] = lbl.transform(train['Street'].values)
test['Street'] = lbl.transform(test['Street'].values)

lbl.fit(list(train['Trap'].values) + list(test['Trap'].values))
train['Trap'] = lbl.transform(train['Trap'].values)
test['Trap'] = lbl.transform(test['Trap'].values)

In [205]:
# drop columns with -1s
train = train.loc[:,(train != -1).any(axis=0)]
test = test.loc[:,(test != -1).any(axis=0)]

In [206]:
# covert all data types to floats
train = train.astype(float)
test = test.astype(float)

In [207]:
## feature engineering
# Temperature difference
train['temp_delta_x'] = train.Tmax_x - train.Tmin_x
train['temp_delta_y'] = train.Tmax_y - train.Tmin_y
train['temp_delta_x'] = train.Tmax_x - train.Tmin_x
train['temp_delta_y'] = train.Tmax_y - train.Tmin_y
test['temp_delta_x'] = test.Tmax_x - test.Tmin_x
test['temp_delta_y'] = test.Tmax_y - test.Tmin_y
test['temp_delta_x'] = test.Tmax_x - test.Tmin_x
test['temp_delta_y'] = test.Tmax_y - test.Tmin_y

In [208]:
# Is raining?
train['israining_x'] = (train.PrecipTotal_x > 0).astype(int)
train['israining_y'] = (train.PrecipTotal_y > 0).astype(int)
test['israining_x'] = (test.PrecipTotal_x > 0).astype(int)
test['israining_y'] = (test.PrecipTotal_y > 0).astype(int)

In [209]:
# Length of sunlight
train['sunlight'] = train.Sunset_x - train.Sunrise_x
test['sunlight'] = test.Sunset_x - test.Sunrise_x

In [210]:
# Replace outliers with median value
train.WetBulb_x = train.WetBulb_x.apply(lambda x: train.WetBulb_x.median() if x < 0 else x)
test.WetBulb_x = test.WetBulb_x.apply(lambda x: test.WetBulb_x.median() if x < 0 else x)

train.StnPressure_x = train.StnPressure_x.apply(lambda x: train.StnPressure_x.median() if x < 0 else x)
test.StnPressure_x = test.StnPressure_x.apply(lambda x: test.StnPressure_x.median() if x < 0 else x)

In [211]:
# Relative humidity approximation
train['RH'] = 100 - (25 / 9) * (train.Tavg_x - train.DewPoint_x)
test['RH'] = 100 - (25 / 9) * (test.Tavg_x - test.DewPoint_x)

In [212]:
# Is heat wave?
train['isheat'] = (train.Heat_x > 0).astype(float)
test['isheat'] = (test.Heat_x > 0).astype(float)

In [213]:
# Vectorize resultant wind into north and east components
wind_dir_north_x = train.ResultDir_x.apply(lambda x: np.sin(np.deg2rad(x)))
wind_dir_east_x = train.ResultDir_x.apply(lambda x: np.cos(np.deg2rad(x)))
train['wind_north_x'] = wind_dir_north_x * train.ResultSpeed_x
train['wind_east_x'] = wind_dir_east_x * train.ResultSpeed_x

wind_dir_north_y = train.ResultDir_y.apply(lambda x: np.sin(np.deg2rad(x)))
wind_dir_east_y = train.ResultDir_y.apply(lambda x: np.cos(np.deg2rad(x)))
train['wind_north_y'] = wind_dir_north_y * train.ResultSpeed_y
train['wind_east_y'] = wind_dir_east_y * train.ResultSpeed_y

wind_dir_north_x = test.ResultDir_x.apply(lambda x: np.sin(np.deg2rad(x)))
wind_dir_east_x = test.ResultDir_x.apply(lambda x: np.cos(np.deg2rad(x)))
test['wind_north_x'] = wind_dir_north_x * test.ResultSpeed_x
test['wind_east_x'] = wind_dir_east_x * test.ResultSpeed_x

wind_dir_north_y = test.ResultDir_y.apply(lambda x: np.sin(np.deg2rad(x)))
wind_dir_east_y = test.ResultDir_y.apply(lambda x: np.cos(np.deg2rad(x)))
test['wind_north_y'] = wind_dir_north_y * test.ResultSpeed_y
test['wind_east_y'] = wind_dir_east_y * test.ResultSpeed_y

In [214]:
# Vectorize average wind into north and east components
train['avg_wind_north_x'] = wind_dir_north_x * train.AvgSpeed_x
train['avg_wind_east_x'] = wind_dir_east_x * train.AvgSpeed_x

train['avg_wind_north_y'] = wind_dir_north_y * train.AvgSpeed_y
train['avg_wind_east_y'] = wind_dir_east_y * train.AvgSpeed_y

test['avg_wind_north_x'] = wind_dir_north_x * test.AvgSpeed_x
test['avg_wind_east_x'] = wind_dir_east_x * test.AvgSpeed_x

test['avg_wind_north_y'] = wind_dir_north_y * test.AvgSpeed_y
test['avg_wind_east_y'] = wind_dir_east_y * test.AvgSpeed_y

In [215]:
# train = pd.concat([train.reset_index(), (pd.get_dummies(train.year)).reset_index(), (pd.get_dummies(train.month)).reset_index(), (pd.get_dummies(train.Species)).reset_index()], axis='columns')
# test = pd.concat([test.reset_index(), (pd.get_dummies(test.year)).reset_index(), (pd.get_dummies(test.month)).reset_index(), (pd.get_dummies(test.Species)).reset_index()], axis='columns')

In [216]:
train = pd.get_dummies(train, columns=['Species'], drop_first=True)
test = pd.get_dummies(test, columns=['Species'], drop_first=True)

In [217]:
drop_list = ['PrecipTotal_x', 'PrecipTotal_y', 'Sunrise_x', 'Sunset_x',
             'Heat_x', 'Depth_x', 'SnowFall_x', 'ResultDir_x', 'ResultSpeed_x',
             'ResultDir_y', 'ResultSpeed_y', 'AvgSpeed_x', 'AvgSpeed_y', 'year',
             'month', 'day']

train.drop(drop_list + ['NumMosquitos'], axis='columns', inplace=True)
test.drop(drop_list + ['Species_7.0'], axis='columns', inplace=True)

In [218]:
# cols_to_use = [
#  'Block',
#  'Street',
#  'Trap',
#  'Latitude',
#  'Longitude',
#  'AddressAccuracy',
#  'day',
#  'Tmax_x',
#  'Tmin_x',
#  'Tavg_x',
#  'Depart_x',
#  'DewPoint_x',
#  'WetBulb_x',
#  'Cool_x',
#  'StnPressure_x',
#  'SeaLevel_x',
#  'Tmax_y',
#  'Tmin_y',
#  'Tavg_y',
#  'DewPoint_y',
#  'WetBulb_y',
#  'Heat_y',
#  'Cool_y',
#  'StnPressure_y',
#  'SeaLevel_y',
#  'temp_delta_x',
#  'temp_delta_y',
#  'israining_x',
#  'israining_y',
#  'sunlight',
#  'RH',
#  'isheat',
#  'wind_north_x',
#  'wind_east_x',
#  'wind_north_y',
#  'wind_east_y',
#  'avg_wind_north_x',
#  'avg_wind_east_x',
#  'avg_wind_north_y',
#  'avg_wind_east_y',
#  6.0,
#  7.0,
#  8.0,
#  9.0,
#  10.0,
#  0.0,
#  1.0,
#  2.0,
#  3.0,
#  4.0,
#  5.0,
#  6.0,
#  7.0]

# train = train[cols_to_use + ['WnvPresent']]
# test = test[cols_to_use]

In [219]:
# Set dependent (y) and indepedent (X) variables
y = train.WnvPresent
X = train.drop('WnvPresent', axis='columns')

In [220]:
# Baseline accuracy
1 - y.mean()

0.9475537787930707

In [221]:
# Scale X for modeling
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [222]:
# Train/test split for validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [223]:
# Random forest
cross_val_score(RandomForestClassifier(max_depth=4, max_features=5), X_train, y_train,
                cv=5, n_jobs=-1, verbose=0).mean()

0.9475250626167533

*Doesn't break baseline. What is happening?*

In [224]:
# Confustion matrix
clf = RandomForestClassifier(max_depth=4, max_features=5)
clf.fit(X_train, y_train)
confusion_matrix(y_test, clf.predict(X_test))

array([[1992,    0],
       [ 110,    0]])

*The model is maximizing accuracy by predicting "No" everytime. This is the result of have unbalanced classes.*

In [225]:
# Dataframe of positive West Nile results
pos_train = train[train.WnvPresent == 1]

In [226]:
# Oversampled unrepresented class
traino = pd.concat([train, pos_train], axis=0)
for _ in range(10):
    traino = pd.concat([traino, pos_train], axis=0)

In [227]:
# Oversampled train/test split
yo = traino.WnvPresent
Xo = traino.drop('WnvPresent', axis='columns')
Xo = pd.DataFrame(scaler.fit_transform(Xo), columns=Xo.columns)
X_traino, X_testo, y_traino, y_testo = train_test_split(Xo, yo, test_size=0.2, stratify=yo)

### We need to maximize recall

In [228]:
clf = RandomForestClassifier(max_depth=4, max_features=7)
clf.fit(X_traino, y_traino)
print(confusion_matrix(y_test, clf.predict(X_test)))
print(classification_report(y_test, clf.predict(X_test), target_names=['No', 'Yes']))

[[1554  438]
 [  38   72]]
             precision    recall  f1-score   support

         No       0.98      0.78      0.87      1992
        Yes       0.14      0.65      0.23       110

avg / total       0.93      0.77      0.83      2102



In [229]:
np.random.seed()

In [230]:
model = Sequential()
input_units = X_train.shape[1]
hidden_units = input_units

model.add(Dense(hidden_units * 2,
                input_dim=input_units,
                activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(hidden_units * 2,
                input_dim=input_units,
                activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10,
                input_dim=input_units,
                activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
train_loss = []
test_loss = []
train_acc = []
test_acc = []

In [ ]:
%%time
hist = model.fit(X_traino, y_traino,
                 validation_data=(X_test, y_test),
                 epochs=750,
                 batch_size=64,
                 shuffle=True)

Train on 13253 samples, validate on 2102 samples
Epoch 1/750
13253/13253 [==============================] - 1s 82us/step - loss: 0.6045 - acc: 0.6613 - val_loss: 0.4571 - val_acc: 0.7840
Epoch 2/750
13253/13253 [==============================] - 0s 34us/step - loss: 0.5088 - acc: 0.7417 - val_loss: 0.4347 - val_acc: 0.7716
Epoch 3/750
13253/13253 [==============================] - 1s 40us/step - loss: 0.4796 - acc: 0.7605 - val_loss: 0.4959 - val_acc: 0.7079
Epoch 4/750
13253/13253 [==============================] - 1s 45us/step - loss: 0.4586 - acc: 0.7732 - val_loss: 0.4712 - val_acc: 0.6970
Epoch 5/750
13253/13253 [==============================] - 1s 45us/step - loss: 0.4436 - acc: 0.7785 - val_loss: 0.4569 - val_acc: 0.7188
Epoch 6/750
13253/13253 [==============================] - 1s 45us/step - loss: 0.4379 - acc: 0.7792 - val_loss: 0.4234 - val_acc: 0.7607
Epoch 7/750
13253/13253 [==============================] - 1s 41us/step - loss: 0.4295 - acc: 0.7810 - val_loss: 0.4209 - v

13253/13253 [==============================] - 1s 40us/step - loss: 0.2765 - acc: 0.8782 - val_loss: 0.3126 - val_acc: 0.8249
Epoch 60/750
13253/13253 [==============================] - 1s 38us/step - loss: 0.2769 - acc: 0.8776 - val_loss: 0.2808 - val_acc: 0.8430
Epoch 61/750
13253/13253 [==============================] - 1s 43us/step - loss: 0.2810 - acc: 0.8764 - val_loss: 0.2882 - val_acc: 0.8359
Epoch 62/750
13253/13253 [==============================] - 1s 51us/step - loss: 0.2817 - acc: 0.8734 - val_loss: 0.3011 - val_acc: 0.8254
Epoch 63/750
13253/13253 [==============================] - 1s 39us/step - loss: 0.2700 - acc: 0.8825 - val_loss: 0.2862 - val_acc: 0.8382
Epoch 64/750
13253/13253 [==============================] - 1s 58us/step - loss: 0.2731 - acc: 0.8775 - val_loss: 0.2805 - val_acc: 0.8440
Epoch 65/750
13253/13253 [==============================] - 0s 31us/step - loss: 0.2711 - acc: 0.8800 - val_loss: 0.2856 - val_acc: 0.8397
Epoch 66/750
13253/13253 [==============

Epoch 118/750
13253/13253 [==============================] - 1s 44us/step - loss: 0.2373 - acc: 0.8999 - val_loss: 0.2552 - val_acc: 0.8663
Epoch 119/750
13253/13253 [==============================] - 0s 34us/step - loss: 0.2377 - acc: 0.8996 - val_loss: 0.2499 - val_acc: 0.8701
Epoch 120/750
13253/13253 [==============================] - 0s 31us/step - loss: 0.2326 - acc: 0.9017 - val_loss: 0.2530 - val_acc: 0.8716
Epoch 121/750
13253/13253 [==============================] - 0s 34us/step - loss: 0.2355 - acc: 0.9009 - val_loss: 0.2537 - val_acc: 0.8744
Epoch 122/750
13253/13253 [==============================] - 0s 35us/step - loss: 0.2411 - acc: 0.8982 - val_loss: 0.2354 - val_acc: 0.8701
Epoch 123/750
13253/13253 [==============================] - 1s 38us/step - loss: 0.2432 - acc: 0.8974 - val_loss: 0.2414 - val_acc: 0.8768
Epoch 124/750
13253/13253 [==============================] - 0s 33us/step - loss: 0.2423 - acc: 0.8981 - val_loss: 0.2428 - val_acc: 0.8739
Epoch 125/750
13253/

13253/13253 [==============================] - 0s 30us/step - loss: 0.2162 - acc: 0.9079 - val_loss: 0.2338 - val_acc: 0.8806
Epoch 177/750
13253/13253 [==============================] - 0s 30us/step - loss: 0.2186 - acc: 0.9096 - val_loss: 0.2237 - val_acc: 0.8773
Epoch 178/750
13253/13253 [==============================] - 0s 30us/step - loss: 0.2098 - acc: 0.9113 - val_loss: 0.2346 - val_acc: 0.8849
Epoch 179/750
13253/13253 [==============================] - 0s 30us/step - loss: 0.2158 - acc: 0.9096 - val_loss: 0.2265 - val_acc: 0.8839
Epoch 180/750
13253/13253 [==============================] - 0s 30us/step - loss: 0.2166 - acc: 0.9090 - val_loss: 0.2499 - val_acc: 0.8716
Epoch 181/750
13253/13253 [==============================] - 0s 36us/step - loss: 0.2116 - acc: 0.9120 - val_loss: 0.2346 - val_acc: 0.8782
Epoch 182/750
13253/13253 [==============================] - 0s 32us/step - loss: 0.2151 - acc: 0.9079 - val_loss: 0.2301 - val_acc: 0.8754
Epoch 183/750
13253/13253 [=======

13253/13253 [==============================] - 0s 34us/step - loss: 0.2055 - acc: 0.9153 - val_loss: 0.2126 - val_acc: 0.8934
Epoch 235/750
13253/13253 [==============================] - 0s 31us/step - loss: 0.2022 - acc: 0.9153 - val_loss: 0.2179 - val_acc: 0.8825
Epoch 236/750
13253/13253 [==============================] - 0s 31us/step - loss: 0.2019 - acc: 0.9136 - val_loss: 0.2171 - val_acc: 0.8839
Epoch 237/750
13253/13253 [==============================] - 0s 29us/step - loss: 0.2064 - acc: 0.9133 - val_loss: 0.2065 - val_acc: 0.8873
Epoch 238/750
13253/13253 [==============================] - 0s 36us/step - loss: 0.2011 - acc: 0.9168 - val_loss: 0.2119 - val_acc: 0.8901
Epoch 239/750
13253/13253 [==============================] - 0s 32us/step - loss: 0.2061 - acc: 0.9144 - val_loss: 0.2193 - val_acc: 0.8768
Epoch 240/750
13253/13253 [==============================] - 0s 30us/step - loss: 0.2074 - acc: 0.9113 - val_loss: 0.2157 - val_acc: 0.8782
Epoch 241/750
13253/13253 [=======

...

13253/13253 [==============================] - 1s 40us/step - loss: 0.1829 - acc: 0.9249 - val_loss: 0.2065 - val_acc: 0.8968
Epoch 467/750
13253/13253 [==============================] - 1s 51us/step - loss: 0.1798 - acc: 0.9278 - val_loss: 0.1968 - val_acc: 0.9058
Epoch 468/750
13253/13253 [==============================] - 0s 36us/step - loss: 0.1821 - acc: 0.9239 - val_loss: 0.2062 - val_acc: 0.8972
Epoch 469/750
13253/13253 [==============================] - 0s 32us/step - loss: 0.1825 - acc: 0.9250 - val_loss: 0.2087 - val_acc: 0.8996
Epoch 470/750
13253/13253 [==============================] - 0s 34us/step - loss: 0.1876 - acc: 0.9212 - val_loss: 0.1952 - val_acc: 0.9010
Epoch 471/750
13253/13253 [==============================] - 1s 46us/step - loss: 0.1791 - acc: 0.9257 - val_loss: 0.1966 - val_acc: 0.9044
Epoch 472/750
13253/13253 [==============================] - 1s 39us/step - loss: 0.1819 - acc: 0.9250 - val_loss: 0.2009 - val_acc: 0.9063
Epoch 473/750
13253/13253 [=======

13253/13253 [==============================] - 1s 59us/step - loss: 0.1802 - acc: 0.9251 - val_loss: 0.1992 - val_acc: 0.9096
Epoch 525/750
13253/13253 [==============================] - 1s 47us/step - loss: 0.1794 - acc: 0.9269 - val_loss: 0.2035 - val_acc: 0.9049
Epoch 526/750
13253/13253 [==============================] - 1s 40us/step - loss: 0.1766 - acc: 0.9260 - val_loss: 0.2040 - val_acc: 0.8991
Epoch 527/750
13253/13253 [==============================] - 0s 32us/step - loss: 0.1733 - acc: 0.9258 - val_loss: 0.2110 - val_acc: 0.8968
Epoch 528/750
13253/13253 [==============================] - 0s 32us/step - loss: 0.1776 - acc: 0.9260 - val_loss: 0.1926 - val_acc: 0.9053
Epoch 529/750
13253/13253 [==============================] - 0s 32us/step - loss: 0.1806 - acc: 0.9256 - val_loss: 0.1952 - val_acc: 0.8977
Epoch 530/750
13253/13253 [==============================] - 0s 33us/step - loss: 0.1738 - acc: 0.9271 - val_loss: 0.1965 - val_acc: 0.9058
Epoch 531/750
13253/13253 [=======

13253/13253 [==============================] - 1s 44us/step - loss: 0.1731 - acc: 0.9282 - val_loss: 0.1874 - val_acc: 0.9129
Epoch 583/750
13253/13253 [==============================] - 1s 49us/step - loss: 0.1866 - acc: 0.9221 - val_loss: 0.1982 - val_acc: 0.9053
Epoch 584/750
13253/13253 [==============================] - 1s 44us/step - loss: 0.1762 - acc: 0.9264 - val_loss: 0.1962 - val_acc: 0.9091
Epoch 585/750
13253/13253 [==============================] - 1s 45us/step - loss: 0.1789 - acc: 0.9254 - val_loss: 0.1928 - val_acc: 0.9044
Epoch 586/750
13253/13253 [==============================] - 1s 49us/step - loss: 0.1793 - acc: 0.9259 - val_loss: 0.1895 - val_acc: 0.9082
Epoch 587/750
13253/13253 [==============================] - 1s 44us/step - loss: 0.1774 - acc: 0.9276 - val_loss: 0.1955 - val_acc: 0.9091
Epoch 588/750
13253/13253 [==============================] - 1s 41us/step - loss: 0.1713 - acc: 0.9311 - val_loss: 0.2004 - val_acc: 0.9063
Epoch 589/750
13253/13253 [=======

13253/13253 [==============================] - 0s 30us/step - loss: 0.1705 - acc: 0.9296 - val_loss: 0.1866 - val_acc: 0.9091
Epoch 641/750
13253/13253 [==============================] - 0s 33us/step - loss: 0.1710 - acc: 0.9288 - val_loss: 0.1870 - val_acc: 0.9096
Epoch 642/750
13253/13253 [==============================] - 0s 32us/step - loss: 0.1709 - acc: 0.9294 - val_loss: 0.2040 - val_acc: 0.9025
Epoch 643/750
13253/13253 [==============================] - 0s 31us/step - loss: 0.1677 - acc: 0.9303 - val_loss: 0.2076 - val_acc: 0.8996
Epoch 644/750
13253/13253 [==============================] - 0s 30us/step - loss: 0.1764 - acc: 0.9274 - val_loss: 0.1854 - val_acc: 0.9058
Epoch 645/750
13253/13253 [==============================] - 0s 32us/step - loss: 0.1718 - acc: 0.9279 - val_loss: 0.1967 - val_acc: 0.9096
Epoch 646/750
13253/13253 [==============================] - 0s 30us/step - loss: 0.1784 - acc: 0.9267 - val_loss: 0.1893 - val_acc: 0.9072
Epoch 647/750
13253/13253 [=======

13253/13253 [==============================] - 1s 48us/step - loss: 0.1717 - acc: 0.9281 - val_loss: 0.1983 - val_acc: 0.9029
Epoch 699/750
13253/13253 [==============================] - 1s 49us/step - loss: 0.1731 - acc: 0.9282 - val_loss: 0.1992 - val_acc: 0.9034
Epoch 700/750
13253/13253 [==============================] - 1s 49us/step - loss: 0.1740 - acc: 0.9251 - val_loss: 0.1951 - val_acc: 0.9049
Epoch 701/750
13253/13253 [==============================] - 1s 46us/step - loss: 0.1692 - acc: 0.9289 - val_loss: 0.1815 - val_acc: 0.9125
Epoch 702/750
13253/13253 [==============================] - 1s 46us/step - loss: 0.1679 - acc: 0.9334 - val_loss: 0.1857 - val_acc: 0.9115
Epoch 703/750
13253/13253 [==============================] - 1s 45us/step - loss: 0.1724 - acc: 0.9278 - val_loss: 0.2010 - val_acc: 0.9001
Epoch 704/750
13253/13253 [==============================] - 1s 41us/step - loss: 0.1757 - acc: 0.9275 - val_loss: 0.1961 - val_acc: 0.9015
Epoch 705/750
13253/13253 [=======

In [ ]:
train_loss += hist.history['loss']
test_loss += hist.history['val_loss']
train_acc += hist.history['acc']
test_acc += hist.history['val_acc']

plt.figure()
plt.plot(train_loss)
plt.plot(test_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best') ## I love this loc = 'best' command.
plt.show()
plt.figure()
plt.plot(train_acc)
plt.plot(test_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()
test_acc[-1]

In [ ]:
roc_auc_score(y_test, model.predict(X_test))

In [ ]:
roc_curve(y_test, model.predict(X_test))

In [ ]:
for threshold in np.linspace(0, 1, 101):
    print('Threshold: {}'.format(threshold))
    print('Confusion Matrix')
    print(confusion_matrix(y_test, (model.predict(X_test) > threshold).astype(float)))
    print(classification_report(y_test, (model.predict(X_test) > threshold).astype(float), target_names=['No', 'Yes']))
    print('*******************************\n')

In [ ]:
y_testo.mean()

In [ ]:
sens = []
spec = []
most_goodest = []
pred = model.predict(X_test)
for n in np.linspace(0, 1, 10001):
    temp = 1 * (pred > n)
    tn, fp, fn, tp = confusion_matrix(y_test, temp).ravel()
    sens += [tp / (tp + fn)]
    spec += [1 - (tn / (tn + fp))]
    most_goodest += [tp + tn]
# sens += [0]
# spec += [0]
plt.plot(spec, sens);
plt.show()
plt.plot(np.linspace(0, 1, 10001), most_goodest)
plt.show()

In [86]:
pred

array([[9.32551871e-21],
       [9.96978164e-01],
       [1.55285333e-11],
       ...,
       [1.26641605e-11],
       [1.39407691e-13],
       [0.00000000e+00]], dtype=float32)

In [62]:
sens, spec

([1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.9994472084024323,
  0.9994496422674739,
  0.9994496422674739,
  0.999450247388675,
  0.999450247388675,
  0.999450247388675,
  0.9994505494505495,
  0.9994520547945206,
  0.9994523548740416,
  0.9994523548740416,
  0.9994532531437944,
  0.9994532531437944,
  0.9994535519125683,
  0.9994541484716157,
  0.9994541484716157,
  0.9994547437295529,
  0.9978354978354979,
  0.9978354978354979,
  0.997836668469443,
  0.9978413383702105,
  0.997843665768194,
  0.9967793880837359,
  0.9967931587386424,
  0.9957356076759062,
  0.9957401490947817,
  0.9952127659574468,
  0.9952153110047847,
  0.9952178533475027,
  0.9952229299363057,
  0.9952229299363057,
  0.9946977730646872,
  0.9946977730646872,
  0.9947201689545935,
  0.9942013705851345,
  0.9936775553213909,
  0.9936808846761453,
  0.9936808846761453,
  0.9936842105263158,
 

In [106]:
max([x for x in zip(most_goodest, np.linspace(0, 1, 10001))])

(2037, 0.9792000000000001)

In [46]:
temp = 1 * (pred > .9792)
tn, fp, fn, tp = confusion_matrix(y_test, temp).ravel()
tn, fp, fn, tp

(1984, 8, 86, 24)

In [75]:
mat[0][1]

120

In [91]:
np.linspace(0, 1, 1001)

array([0.   , 0.001, 0.002, ..., 0.998, 0.999, 1.   ])

In [112]:
sample.WnvPresent

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
116263    0
116264    0
116265    0
116266    0
116267    0
116268    0
116269    0
116270    0
116271    0
116272    0
116273    0
116274    0
116275    0
116276    0
116277    0
116278    0
116279    0
116280    0
116281    0
116282    0
116283    0
116284    0
116285    0
116286    0
116287    0
116288    0
116289    0
116290    0
116291    0
116292    0
Name: WnvPresent, Length: 116293, dtype: int64

In [ ]:
preds = model.predict(test)
submission = pd.DataFrame([[i,x[0]] for i, x in enumerate(preds.tolist())], columns=sample.columns)

In [131]:
submission.to_csv('submission.csv')

In [133]:
submission.shape

(2102, 2)

In [86]:
exclusions = [x for x in train.columns if x not in test.columns] + [x for x in test.columns if x not in train.columns]

In [87]:
exclusions

['NumMosquitos', 'WnvPresent', 'Id']

In [160]:
beep = train.drop(columns=exclusions)
boop = test.drop(columns=exclusions)

In [178]:
boop.index

RangeIndex(start=0, stop=116293, step=1)

In [193]:
trainer = train.drop(columns='WnvPresent')

In [194]:
trainer.columns

Index(['Block', 'Street', 'Trap', 'Latitude', 'Longitude', 'AddressAccuracy',
       'Tmax_x', 'Tmin_x', 'Tavg_x', 'Depart_x', 'DewPoint_x', 'WetBulb_x',
       'Cool_x', 'StnPressure_x', 'SeaLevel_x', 'Tmax_y', 'Tmin_y', 'Tavg_y',
       'DewPoint_y', 'WetBulb_y', 'Heat_y', 'Cool_y', 'StnPressure_y',
       'SeaLevel_y', 'temp_delta_x', 'temp_delta_y', 'israining_x',
       'israining_y', 'sunlight', 'RH', 'isheat', 'wind_north_x',
       'wind_east_x', 'wind_north_y', 'wind_east_y', 'avg_wind_north_x',
       'avg_wind_east_x', 'avg_wind_north_y', 'avg_wind_east_y', 'Species_1.0',
       'Species_2.0', 'Species_3.0', 'Species_4.0', 'Species_5.0',
       'Species_6.0'],
      dtype='object')

In [189]:
test.columns

Index(['Block', 'Street', 'Trap', 'Latitude', 'Longitude', 'AddressAccuracy',
       'Tmax_x', 'Tmin_x', 'Tavg_x', 'Depart_x', 'DewPoint_x', 'WetBulb_x',
       'Cool_x', 'StnPressure_x', 'SeaLevel_x', 'Tmax_y', 'Tmin_y', 'Tavg_y',
       'DewPoint_y', 'WetBulb_y', 'Heat_y', 'Cool_y', 'StnPressure_y',
       'SeaLevel_y', 'temp_delta_x', 'temp_delta_y', 'israining_x',
       'israining_y', 'sunlight', 'RH', 'isheat', 'wind_north_x',
       'wind_east_x', 'wind_north_y', 'wind_east_y', 'avg_wind_north_x',
       'avg_wind_east_x', 'avg_wind_north_y', 'avg_wind_east_y', 'Species_1.0',
       'Species_2.0', 'Species_3.0', 'Species_4.0', 'Species_5.0',
       'Species_6.0', 'Species_7.0'],
      dtype='object')

In [196]:
[x for x,y in zip(trainer.columns, test.columns) if x==y]

['Block',
 'Street',
 'Trap',
 'Latitude',
 'Longitude',
 'AddressAccuracy',
 'Tmax_x',
 'Tmin_x',
 'Tavg_x',
 'Depart_x',
 'DewPoint_x',
 'WetBulb_x',
 'Cool_x',
 'StnPressure_x',
 'SeaLevel_x',
 'Tmax_y',
 'Tmin_y',
 'Tavg_y',
 'DewPoint_y',
 'WetBulb_y',
 'Heat_y',
 'Cool_y',
 'StnPressure_y',
 'SeaLevel_y',
 'temp_delta_x',
 'temp_delta_y',
 'israining_x',
 'israining_y',
 'sunlight',
 'RH',
 'isheat',
 'wind_north_x',
 'wind_east_x',
 'wind_north_y',
 'wind_east_y',
 'avg_wind_north_x',
 'avg_wind_east_x',
 'avg_wind_north_y',
 'avg_wind_east_y',
 'Species_1.0',
 'Species_2.0',
 'Species_3.0',
 'Species_4.0',
 'Species_5.0',
 'Species_6.0']